In [5]:

from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=1)
'''
GET READY 
'''
from eden.converter.fasta import fasta_to_sequence
import itertools
def rfam_uri(family_id):
    return 'http://rfam.xfam.org/family/%s/alignment?acc=%s&format=fastau&download=0'%(family_id,family_id)
def rfam_uri(family_id):
    return '%s.fa'%(family_id)
 
def get_graphss(rfam_id = '../toolsdata/RF00005'):
    return fasta_to_sequence(rfam_uri(rfam_id))

def get_graphs(rfam_id = '../toolsdata/RF00005', count=100):
    for a,b in itertools.islice( get_graphss(rfam_id),count):
        yield b

from eden.converter.fasta import fasta_to_sequence
def get_sequences(size=9999):
    sequences = itertools.islice( fasta_to_sequence("../toolsdata/RF00005.fa"), size)
    return [ b for (a,b) in sequences ]

def get_sequences_with_names(size=9999):
    sequences = itertools.islice( fasta_to_sequence("../toolsdata/RF00005.fa"), size)
    return sequences


# imports for later
import graphlearn.abstract_graphs.RNA as rna
from  graphlearn.feasibility import FeasibilityChecker as Checker
from graphlearn.estimator import Wrapper
import numpy
from graphlearn.utils import evaltools
from eden.util import random_bipartition_iter
import random
import networkx as nx
class MyWrapper(Wrapper):
    def unwrap(self,graphmanager):

        if type(graphmanager)==nx.Graph or type(graphmanager)==nx.DiGraph:
            graph=graphmanager.copy()

        else:
            graph = graphmanager.graph( nested=False  ).copy()

        if type(graph) == nx.DiGraph:
            graph=nx.Graph(graph)
        return graph


def wrap_fraction(fraction=0.1,repeats=2):
    # choosing some graphs, 
    # having array to save results

    
    for i in range(repeats):
        badscores=[]
        goodscores=[]
        graphs = get_sequences_with_names(size=923)
        graphs,not_used = random_bipartition_iter(graphs,fraction,random_state=random.random()*i*1000)
        estimator=MyWrapper( nu=.27, cv=3, n_jobs=-1)
        sampler=rna.AbstractSampler(radius_list=[0,1],
                                    thickness_list=[2], 
                                    min_cip_count=1, 
                                    min_interface_count=2, 
                                    preprocessor=rna.PreProcessor(base_thickness_list=[1],ignore_inserts=True), 
                                    postprocessor=rna.PostProcessor(),
                                    estimator=estimator)
        sampler.preprocessor.set_param(sampler.vectorizer)
        graphmanagers = sampler.preprocessor.fit_transform(graphs)
        sampler.estimatorobject.fit(graphmanagers,vectorizer=sampler.vectorizer,
                                    random_state=sampler.random_state)

        #test
        for graphman in graphmanagers:
            struct = evaltools.dotbracket_to_shape(graphman.structure,shapesversion=3)
            score =  sampler.estimatorobject.score(graphman)
            if struct=="[[][][]]":
                goodscores.append(score)
            else:
                badscores.append(score)

        print "afraction=%f , instances=%f, good=%d , bad=%d" % (fraction,fraction*923,len(goodscores),len(badscores))
        a= numpy.array(badscores)
        print 'bad:mean/std ',numpy.mean(a, axis=0),' ',numpy.std(a, axis=0)
        
        a= numpy.array(goodscores)
        print 'cgood:mean/std ',numpy.mean(a, axis=0),' ',numpy.std(a, axis=0)
        
        a= numpy.array(goodscores+badscores)
        print 'dbad+good:mean/std ',numpy.mean(a, axis=0),' ',numpy.std(a, axis=0)
        print ''
        
        
        
wrap_fraction(fraction=0.05,repeats=5)
        


afraction=0.050000 , instances=46.150000, good=31 , bad=15
bad:mean/std  0.354876955003   0.292545380381
cgood:mean/std  0.908203682896   0.0434262376759
dbad+good:mean/std  0.727771054235   0.310581049147

afraction=0.050000 , instances=46.150000, good=34 , bad=12
bad:mean/std  0.251922355768   0.285373473068
cgood:mean/std  0.866642645427   0.105708070543
dbad+good:mean/std  0.706280830734   0.319946305947

afraction=0.050000 , instances=46.150000, good=29 , bad=17
bad:mean/std  0.369462120274   0.284702397531
cgood:mean/std  0.92223014636   0.0416923332098
dbad+good:mean/std  0.717946310633   0.319751180879

afraction=0.050000 , instances=46.150000, good=36 , bad=10
bad:mean/std  0.150016536563   0.232128502558
cgood:mean/std  0.853872750209   0.20539654493
dbad+good:mean/std  0.700860529852   0.359188238022

afraction=0.050000 , instances=46.150000, good=26 , bad=20
bad:mean/std  0.41102982848   0.26927337632
cgood:mean/std  0.936364286161   0.0360928282491
dbad+good:mean/std  0.70

In [4]:
from eden.util import random_bipartition_iter
import random
iterable=xrange(20)
for a in range(5):
    a,b=random_bipartition_iter(iterable, relative_size=.2, random_state=random.random())
    print list(a)

[1, 5, 9, 17]
[3, 5, 9, 14]
[1, 3, 7, 11]
[5, 6, 10, 13]
[0, 1, 7, 8]


In [15]:
evaltools.dotbracket_to_shape('(((...(((...))))))',shapesversion=3)

'[[]]'